<a href="https://colab.research.google.com/github/Allen001822480/Info7374SpringTeam5/blob/Assignment4/Assignment4_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import theano
import os.path
import sys
import nltk
import re
import time
import nltk
import _pickle
nltk.download('punkt')

from keras.utils import plot_model

word_embedding_size = 100
sentence_embedding_size = 300
dictionary_size = 10000
maxlen_input = 50

vocabulary_file = 'vocabulary_twitter'
weights_file = 'assignment4_twitter.h5'
unknown_token = 'something'
file_saved_context = 'saved_context'
file_saved_answer = 'saved_answer'
name_of_computer = 'team5_chatbot'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using TensorFlow backend.


In [0]:
def greedy_decoder(input):

    flag = 0
    prob = 1
    ans_partial = np.zeros((1,maxlen_input))
    ans_partial[0, -1] = 0  #  the index of the symbol STA (begin of sentence)
    for k in range(maxlen_input - 1):
        ye = model.predict([input, ans_partial])
        yel = ye[0,:]
        p = np.max(yel)
        mp = np.argmax(ye)
        ans_partial[0, 0:-1] = ans_partial[0, 1:]
        ans_partial[0, -1] = mp
        if mp == 1:  #  he index of the symbol END (end of sentence)
            flag = 1
        if flag == 0:    
            prob = prob * p
    text = ''
    for k in ans_partial[0]:
        k = k.astype(int)
        if k < (dictionary_size-2):
            w = vocabulary[k]
            text = text + w[0] + ' '
    return(text, prob)

In [0]:
def preprocess(raw_word, name):
    
    l1 = ['won’t','won\'t','wouldn’t','wouldn\'t','’m', '’re', '’ve', '’ll', '’s','’d', 'n’t', '\'m', '\'re', '\'ve', '\'ll', '\'s', '\'d', 'can\'t', 'n\'t', 'B: ', 'A: ', ',', ';', '.', '?', '!', ':', '. ?', ',   .', '. ,', 'SAT', 'END', 'sta', 'end','ight']
    l2 = ['will not','will not','would not','would not',' am', ' are', ' have', ' will', ' is', ' had', ' not', ' am', ' are', ' have', ' will', ' is', ' had', 'can not', ' not', '', '', ' ,', ' ;', ' .', ' ?', ' !', ' :', '? ', '.', ',', '', '', '', '','']
    l3 = ['-', '_', ' *', ' /', '* ', '/ ', '\"', ' \\"', '\\ ', '--', '...', '. . .']
 
    raw_word = raw_word.lower()
    raw_word = raw_word.replace(', ' + name_of_computer, '')
    raw_word = raw_word.replace(name_of_computer + ' ,', '')

    for j, term in enumerate(l1):
        raw_word = raw_word.replace(term,l2[j])
        
    for term in l3:
        raw_word = raw_word.replace(term,' ')
    
    for j in range(30):
        raw_word = raw_word.replace('. .', '')
        raw_word = raw_word.replace('.  .', '')
        raw_word = raw_word.replace('..', '')
       
    for j in range(5):
        raw_word = raw_word.replace('  ', ' ')
        
    if raw_word[-1] !=  '!' and raw_word[-1] != '?' and raw_word[-1] != '.' and raw_word[-2:] !=  '! ' and raw_word[-2:] != '? ' and raw_word[-2:] != '. ':
        raw_word = raw_word + ' .'
    
    if raw_word == ' !' or raw_word == ' ?' or raw_word == ' .' or raw_word == ' ! ' or raw_word == ' ? ' or raw_word == ' . ':
        raw_word = 'what ?'
    
    if raw_word == '  .' or raw_word == ' .' or raw_word == '  . ':
        raw_word = 'i do not want to talk about it .'
      
    return raw_word

In [0]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
def tokenize(sentences):

    # Tokenizing the sentences into words:
    tokenized_sentences = nltk.word_tokenize(sentences)
    index_to_word = [x[0] for x in vocabulary]
    word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])
    tokenized_sentences = [w if w in word_to_index else unknown_token for w in tokenized_sentences]
    X = np.asarray([word_to_index[w] for w in tokenized_sentences])
    s = X.size
    Q = np.zeros((1,maxlen_input))
    if s < (maxlen_input + 1):
        Q[0,:s] = X
    else:
        Q[0,:] = X[:maxlen_input]
    analyzer = SentimentIntensityAnalyzer()
    score = analyzer.polarity_scores(sentences)
    pos = score['pos']
    Q = Q*pos
    
    return Q

In [0]:
from keras.layers import Input, Embedding, LSTM, Dense, RepeatVector, Dropout, merge
from keras.optimizers import Adam 
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.preprocessing import sequence
from keras.layers import concatenate
ad = Adam(lr=0.00005) 

input_context = Input(shape=(maxlen_input,), dtype='int32', name='context_text')
input_answer = Input(shape=(maxlen_input,), dtype='int32', name='answer_text')
LSTM_encoder = LSTM(sentence_embedding_size, kernel_initializer= 'lecun_uniform', name='Encode_context')
LSTM_decoder = LSTM(sentence_embedding_size, kernel_initializer= 'lecun_uniform', name='Encode_answer')
if os.path.isfile(weights_file):
    Shared_Embedding = Embedding(output_dim=word_embedding_size, input_dim=dictionary_size, input_length=maxlen_input, name='Shared')
else:
    Shared_Embedding = Embedding(output_dim=word_embedding_size, input_dim=dictionary_size, weights=[embedding_matrix], input_length=maxlen_input, name='Shared')
word_embedding_context = Shared_Embedding(input_context)
context_embedding = LSTM_encoder(word_embedding_context)

word_embedding_answer = Shared_Embedding(input_answer)
answer_embedding = LSTM_decoder(word_embedding_answer)

merge_layer = concatenate([context_embedding, answer_embedding], axis=1)
out = Dense(int(dictionary_size/2), activation="relu")(merge_layer)
out = Dense(dictionary_size, activation="softmax")(out)

model = Model(inputs=[input_context, input_answer], outputs = [out])

model.compile(loss='categorical_crossentropy', optimizer=ad)

#plot_model(model, to_file='model_graph.png')    

if os.path.isfile(weights_file):
    model.load_weights(weights_file)


# Loading the data:
vocabulary = _pickle.load(open(vocabulary_file, 'rb'))


In [0]:
import numpy as np
print("\n \n \n \n    CHAT:     \n \n")
prob = 0
que = ''
last_query  = ' '
last_last_query = ''
text = ' '
last_text = ''
print('computer: hi ! please type your name.\n')
name = input('user: ')
print('computer: hi , ' + name +' ! My name is ' + name_of_computer + '.\n') 


while que != 'exit .':
    
    que = input('user: ')
    que = preprocess(que, name_of_computer)
    # Collecting data for training:
    q = last_query + ' ' + text
    a = que
    # Composing the context:
    if prob > 0.2:
        query = text + ' ' + que
    else:    
        query = que
   
    last_text = text
    
    Q = tokenize(query)
    
    # Using the trained model to predict the answer:
    
    predout, prob = greedy_decoder(Q[0:1])
    start_index = predout.find('sta')
    text = preprocess(predout[0:start_index], name)
    print ('computer: ' + text + '    (with probability of %f)'%prob)
    
    last_last_query = last_query    
    last_query = que



 
 
 
    CHAT:     
 

computer: hi ! please type your name.

user: yi
computer: hi , yi ! My name is team5_chatbot.

user: there was a greasy kid at highline who was basically miles dipped in a bucket of ranch
computer:  i am just trying to convince you and wake up having in 7 minutes in the am . . shit do not expect me down  .    (with probability of 0.000001)
user: i killed a mouse today!  
computer:  i am gonna call it in the first part of you know you expect you will ? and you can tell them you feel ?     (with probability of 0.000000)
user: so much to do before leaving for d.c. ahhh
computer:  i do not know it was either or so i can not get of the benefits of the week and all defense and keep all . all . all . it ? ?     (with probability of 0.000000)
user: whatever, i'm gonna uncork this bottle and have me a glass of wine.. 
computer:  wyd you know me then lets me get out there ? is where you know not listen to see again ?     (with probability of 0.097071)
user: i don't alway